In [3]:
import yfinance as yf
from myUtils import yf_download_AdjOHLCV, pickle_dump, pickle_load  # NOQA
verbose = False  # True prints more output

# path to file directories
path_dir = "C:/Users/ping/MyDrive/stocks/MktCap2b_AUMtop1200/"
path_data_dump = path_dir + "VSCode_dump/"
path_symbols_file = path_dir + "source/"

In [4]:
# file names
filename_symbols = path_symbols_file + '2021_Top1200_MktCap_n_AUM.txt'  # symbols text file
filename_pickled_df_OHLCV_all_dates = 'df_OHLCV_all_dates'  # pickled filename
filename_pickled_df_OHLCV_NYSE_dates = 'df_OHLCV_NYSE_dates'  # pickled filename
filename_pickled_symbols_df_OHLCV_NYSE_dates =  'symbols_df_OHLCV_NYSE_dates'  # pickled filename

# filename_symbols = path_symbols_file + "test_symbols_no_XOM.txt"  # symbols text file
# filename_pickled_df_OHLCV_all_dates = "df_test_all_dates"  # pickled filename
# filename_pickled_df_OHLCV_NYSE_dates = "df_test_NYSE_dates"  # pickled filename
# filename_pickled_symbols_df_OHLCV_NYSE_dates = "symbols_df_test_NYSE_dates"  # pickled filename

In [ ]:

def plot_symbols(symbols, path_data_dump, date_start_limit=None,
                 date_end_limit=None, iloc_offset=None):
    """Plots symbol in symbols list. Program ps1n2 created dictionary
    dfs_symbols_OHLCV_download which has symbols' OHLCV data.
    The dictionary is stored in directory path_data_dump. This program plots
    the symbols using data from the dictionary.

    (Copied from def dates_within_limits)
    Order of hierarchy to determine Start and End dates:
    1. given date if it is not None
    2. iloc_offset if it is not None

    start = date_start_limit
    end = date_end_limit
    date_start_limit sets the upper (i.e. oldest) limit.
    date_end_limit sets the lower (i.e. newest) limit.
    date_start is the oldest index date that is within the limits.
    date_end is the newest index date that is within the limits.

    START   END    ILOC-OFFSET  Return->  START DATE             END DATE
    ---------------------------------------------------------------------------
    start   end	   iloc_offset	          date_start	         date_end
    start   end	   none                   date_start	         date_end
    start   none   iloc_offset	          date_start	         date_start+iloc_offset  # NOQA
    start   none   none                   date_start	         index[-1]
    none	end	   iloc_offset	          date_end-iloc_offset   date_end
    none	end	   none                   index[0]	             date_end
    none	none   iloc_offset	          index[0]	             index[-1]
    none	none   none                   index[0]	             index[-1]

    Args:
        symbols(list[str]): list of symbols, e.g. ['AAPL', 'GOOGL', ...]
        path_symbol_data(str): directory path of the symbol data
        date_start_limit(str): limit on the start date in format 'yyyy-mm-dd'
        date_end_limit(str): limit on the end date in format 'yyyy-mm-dd'
        iloc_offset(int): index offset
    
    Return:
        cache_returned_by_plot:
            symbol(str): stock symbol, e.g. 'AMZN'
            date(str): date, e.g. '2018-01-05'
            UI_MW_short[-1](float): last value of UI_MW_short,
                Ulcer-Index / Moving-Window-Short
            UI_MW_long[-1](float): last value of UI_MW_long
                Ulcer-Index / Moving-Window-Long
            diff_UI_MW_short[-1](float): last value of diff_UI_MW_short
                difference of last two 'Ulcer-Index / Moving-Window-Short'
            diff_UI_MW_long[-1](float): last value of diff_UI_MW_long
                difference of last two 'Ulcer-Index / Moving-Window-Long'
    """

    from myUtils import pickle_load, dates_within_limits
    from CStick_DD_OBV_UIMW_Diff_UIMW_cache import candlestick

    print('+'*15 + '  ps4_plot_symbols(symbols, path_data_dump)  ' + '+'*15
          + '\n')

    # directories and file names



    # dfs_filename = 'dfs_symbols_OHLCV_download'
    filename_pickled_symbols_df_OHLCV_NYSE_dates =  'symbols_df_OHLCV_NYSE_dates'  # pickled filename
    
    
    
    # index_symbol = 'XOM'
    # list of cache. Stores cache returned by candlestick plot
    caches_returned_by_plot = []



    # read dfs
    # dfs = pickle_load(path_data_dump, dfs_filename)
    df = pickle_load(path_data_dump, filename_pickled_df_OHLCV_NYSE_dates)



    # date_index_all_dates = dfs[index_symbol].index
    date_index_all_dates = df.index



    # get iloc of date_end_limit in XOM's date index
    date_start, date_end, iloc_date_start, iloc_date_end = \
        dates_within_limits(date_index_all_dates, date_start_limit,
                            date_end_limit, iloc_offset)

    for symbol in symbols:
        df = dfs[symbol]
        df = df[date_start:date_end]
        print('plot symbol {} from {} to {}\n'
              .format(symbol, date_start, date_end))
        # cache_returned_by_plot = candlestick(symbol, df, plot_chart=False)
        cache_returned_by_plot = candlestick(symbol, df, plot_chart=True)
        caches_returned_by_plot.append(cache_returned_by_plot)

    print('{}\n'.format('-'*78))

    return caches_returned_by_plot



In [8]:
date_end_limit = '2022-08-03'
date_start_limit=None
symbols = ['FTEC', 'BCI', 'BTC-USD', 'ETH-USD']
iloc_offset = 252  # number of days to plot

from myUtils import pickle_load, dates_within_limits
from CStick_DD_OBV_UIMW_Diff_UIMW_cache import candlestick

filename_pickled_symbols_df_OHLCV_NYSE_dates =  'symbols_df_OHLCV_NYSE_dates'  # pickled filename
caches_returned_by_plot = []
df = pickle_load(path_data_dump, filename_pickled_df_OHLCV_NYSE_dates)
date_index_all_dates = df.index
date_start, date_end, iloc_date_start, iloc_date_end = \
    dates_within_limits(date_index_all_dates, date_start_limit,
                        date_end_limit, iloc_offset)

In [10]:
print(f'date_start: {date_start}')
print(f'date_end: {date_end}')
print(f'iloc_date_start: {iloc_date_start}')
print(f'iloc_date_end: {iloc_date_end}')
print(f'len(date_index_all_dates): {len(date_index_all_dates)}')

date_start: 2021-08-03 00:00:00
date_end: 2022-08-03 00:00:00
iloc_date_start: 14999
iloc_date_end: 15251
len(date_index_all_dates): 15252


In [13]:
for symbol in symbols:
    df1 = df[symbol]
    df1 = df[date_start:date_end]
    print('plot symbol {} from {} to {}\n'
          .format(symbol, date_start, date_end))
    # cache_returned_by_plot = candlestick(symbol, df, plot_chart=False)
    cache_returned_by_plot = candlestick(symbol, df, plot_chart=True)
    caches_returned_by_plot.append(cache_returned_by_plot)

plot symbol FTEC from 2021-08-03 00:00:00 to 2022-08-03 00:00:00



AttributeError: 'DataFrame' object has no attribute 'close'

In [ ]:

def yf_symbols_close(
    path_dir,
    path_data_dump,
    filename_pickled_df_OHLCV,
    verbose=False,
):
    """Reads adjusted OHLCV data for symbols in pickled df_OHLCV and gathers
       symbols' Close into df_symbols_close. It has a date index and the
       column names are the symbols. Rows and columns with all NaN are dropped.

       Usage: df_symbols_close['SPY'] returns a series with date index and
       SPY Close

    Args:
        path_dir(str): directory path of df_OHLCV
        path_data_dump(str): directory of pickled files
        filename_pickled_df_OHLCV(str): filename of df_OHLCV
        verbose(bool): prints output if True, default False

    Return:
        df_symbols_close(dataframe): dataframe with a date index and
            columns of symbols' Close, column names are symbols
        dates_dropped(DatetimeIndex): dates where all symbols' Close are NaN
        symbols_OHLCV(list): symbols in df_OHLCV
        symbols_dropped(list): symbols where their Close is all NaN
    """

    import pandas as pd
    from myUtils import pickle_load, pickle_dump

    df = pickle_load(
        path_data_dump, filename_pickled_df_OHLCV, verbose=verbose
    )
    # get list of symbols in df_OHLCV, list(df) is a list of tuples
    # e.g.: [('AAPL', 'Open')..('AAPL', 'Volume'),...
    #        ('ZZZZ', 'Open')..('ZZZZ', 'Volume')]
    symbols_OHLCV = list(set([i[0] for i in list(df)]))

    # write symbols' Close to df_symbols_close
    for sym in symbols_OHLCV:
        if (
            sym == symbols_OHLCV[0]
        ):  # create dataframe using the 1st symbol's Close
            df_symbols_close = pd.DataFrame(df[symbols_OHLCV[0]].Close)
        else:  # concatenate Close on subsequent symbols
            df_symbols_close = pd.concat(
                [df_symbols_close, df[sym].Close], axis=1
            )
    # rename column names from Close to symbol names
    df_symbols_close.set_axis(symbols_OHLCV, axis=1, inplace=True)

    # drop rows and columns with all NaN
    print(f"df_symbols_close.info before dropna:\n{df_symbols_close.info()}")
    df_symbols_close_index_before_dropna = df_symbols_close.index
    # drop all NaN rows
    df_symbols_close = df_symbols_close.dropna(
        how="all", axis="index"
    )
    # drop all NaN columns
    df_symbols_close = df_symbols_close.dropna(
        how="all", axis="columns"
    )
    print(f"df_symbols_close.info after dropna:\n{df_symbols_close.info()}")
    # dates with all NaN in row
    dates_dropped = df_symbols_close_index_before_dropna.difference(
        df_symbols_close.index
    )
    # symbols (i.e. column names) with all NaN in column
    symbols_dropped = list(set(symbols_OHLCV) - set(list(df_symbols_close)))

    pickle_dump(
        df_symbols_close, path_data_dump, "df_symbols_close", verbose=verbose
    )
    pickle_dump(
        dates_dropped,
        path_data_dump,
        "df_OHLCV_dates_dropped",
        verbose=verbose,
    )
    pickle_dump(
        symbols_dropped,
        path_data_dump,
        "df_OHLCV_symbols_dropped",
        verbose=verbose,
    )

    return df_symbols_close, dates_dropped, symbols_OHLCV, symbols_dropped
